In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score

# Q1: Import the dataset and examine the variables
# Load the dataset
url = "https://drive.google.com/uc?export=download&id=1Q4J8KS1wm4-_YTuc389enPh6O-eTNcx2"
data = pd.read_csv(url)

# Display the first few rows of the dataset
print(data.head())

# Descriptive statistics
print(data.describe())

# Visualizations
# Distribution of numerical features
data.hist(bins=30, figsize=(15, 10))
plt.show()

# Correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(data.corr(), annot=True, cmap='coolwarm')
plt.show()

# Q2: Preprocess the data
# Check for missing values
print(data.isnull().sum())

# Remove outliers (optional step, can use IQR method or z-score method)
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

for column in ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']:
    data = remove_outliers(data, column)

# No categorical variables to transform

# Q3: Split the dataset into a training set and a test set
X = data.drop('Outcome', axis=1)
y = data['Outcome']

# Use a random seed for reproducibility
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Q4: Train a decision tree model
# Using GridSearchCV to optimize hyperparameters
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 5, 7, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5, 10]
}

dt = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(estimator=dt, param_grid=param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# Best parameters
print("Best parameters found: ", grid_search.best_params_)

# Train the decision tree with best parameters
best_dt = grid_search.best_estimator_
best_dt.fit(X_train, y_train)

# Q5: Evaluate the performance of the decision tree model
y_pred = best_dt.predict(X_test)

# Metrics
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Precision: ", precision_score(y_test, y_pred))
print("Recall: ", recall_score(y_test, y_pred))
print("F1 Score: ", f1_score(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# ROC Curve
y_pred_prob = best_dt.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = roc_auc_score(y_test, y_pred_prob)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

# Q6: Interpret the decision tree
plt.figure(figsize=(20, 10))
plot_tree(best_dt, feature_names=X.columns, class_names=['Non-Diabetic', 'Diabetic'], filled=True)
plt.show()

# Feature importance
feature_importance = pd.DataFrame(best_dt.feature_importances_, index=X.columns, columns=['importance']).sort_values('importance', ascending=False)
print(feature_importance)

# Q7: Validate the decision tree model
# Applying the model to new data
new_data = X_test.sample(5, random_state=42)
new_predictions = best_dt.predict(new_data)
print("New data predictions: ", new_predictions)

# Sensitivity analysis and scenario testing can be conducted by simulating various input scenarios and observing model behavior.
}
